In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, LineString
from datetime import datetime

## Read shapefiles into variables

In [13]:
# Read shape files into geopandas
right_half_tri_SF = gpd.read_file("shapefiles/right_half_triangle/right_half_triangle.shp")[['Name', 'geometry']]
left_half_tri_SF = gpd.read_file("shapefiles/left_half_triangle/left_half_triangle.shp")[['Name', 'geometry']]
island_SF = gpd.read_file("shapefiles/island/island.shp")[['Name', 'geometry']]
newport_SF = gpd.read_file("shapefiles/inundation_map/inundation_map.shp")[['Name', 'geometry']]
# Read Street Shape Files
balboa_blvd = gpd.read_file("shapefiles/Streets/Balboa/Balboa_Blvd-polygon.shp")[['Name', 'geometry']]
w_balboa_blvd = gpd.read_file("shapefiles/Streets/W_Balboa/W_Balboa-polygon.shp")[['Name', 'geometry']]
newport = gpd.read_file("shapefiles/Streets/Newport/Newport_Blvd-polygon.shp")[['Name', 'geometry']]

In [14]:
section_shapefile_list = [right_half_tri_SF,left_half_tri_SF,island_SF,newport_SF]
street_shapefile_list = [balboa_blvd,w_balboa_blvd,newport]

In [62]:
for i in range(len(section_shapefile_list)):

    print(section_shapefile_list[i]['Name'][0])

right_half_triangle
left_half_triangle
Island
Inundation Map


## Read data for each day

In [2]:
# Assign data for each day
d_1_07012018 = 'Data/Probe_Data/2018_07_01_NewportBeach_basic_probe.csv'
d_2_07022018 = 'Data/Probe_Data/2018_07_02_NewportBeach_basic_probe.csv'
d_3_07032018 = 'Data/Probe_Data/2018_07_03_NewportBeach_basic_probe.csv'
d_4_07042018 = 'Data/Probe_Data/2018_07_04_NewportBeach_basic_probe.csv'
d_5_07052018 = 'Data/Probe_Data/2018_07_05_NewportBeach_basic_probe.csv'
d_6_07062018 = 'Data/Probe_Data/2018_07_06_NewportBeach_basic_probe.csv'
d_7_07072018 = 'Data/Probe_Data/2018_07_07_NewportBeach_basic_probe.csv'

In [77]:
data = pd.read_csv(d_4_07042018,
                   names = ["PROBE_ID","SAMPLE_DATE", "LAT", "LONG", "HEADING",\
                           "SPEED","PROBE_DATA_PROVIDER","X","Y", "LOCAL_TIME"])
# This now sorts in date order
data.sort_values(by='SAMPLE_DATE', inplace=True, ascending=True) 
data.head(3)

,PROBE_ID,SAMPLE_DATE,LAT,LONG,HEADING,SPEED,PROBE_DATA_PROVIDER,X,Y,LOCAL_TIME
480,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:00,33.615262,-117.927329,34.0,14.0,CONSUMER14,413977.562793,3.719884e+06,2018-07-03 17:00:00 -0700
479,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:01,33.615269,-117.927325,19.0,12.0,CONSUMER14,413977.940815,3.719885e+06,2018-07-03 17:00:01 -0700
445,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:02,33.615287,-117.927314,19.0,17.0,CONSUMER14,413978.979134,3.719886e+06,2018-07-03 17:00:02 -0700


# Analyze Data

In [90]:
class Analyze_Data:
    
    def __init__(self, data_df,section_shapefile_list,street_shapefile_list):
        # Reading data dataframe into class
        self.data = data_df 
        
        # Read shapefiles into class as lists
        self.section = section_shapefile_list
        self.street = street_shapefile_list
        
    
    def plot(self):
        # Plot Data
        plt.figure(figsize = (12,8))
        plt.scatter(self.data.LONG, self.data.LAT, s = 0.5, c = 'k')
        plt.show()
    
    
    def create_point(self,row):
        # Helper function for turning coordinates into shapely points
        return Point(row['LONG'],row['LAT'])
    
    def append_points(self):

        self.data['coord'] = self.data.apply(self.create_point, axis=1)
        self.data['section'] = 'N/A'
        
    def get_section(self):
        # For separating data into sections based on defined shapefiles
        
        self.append_points()
        
        section_list = []
        
        for i in self.data.index.values:
            if self.section[0]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('right_half_tri')
            elif self.section[1]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('left_half_tri')
            elif self.section[2]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('island')
            elif self.section[3]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('rest_of_new_port')
            else:
                section_list.append('N/A')
        
        self.data['section'] = section_list
        
        return self.data
    
    def get_street(self):
        # For separating data into streets based on defined shapefiles
        
        self.get_section()
        street_list = []

        for i in self.data.index.values:
            if self.street[0]['geometry'][0].contains(self.data['coord'][i]):
                street_list.append('balboa')
            elif self.street[1]['geometry'][0].contains(self.data['coord'][i]):
                street_list.append('w_balboa')
            elif self.street[2]['geometry'][0].contains(self.data['coord'][i]):
                street_list.append('newport')
            else:
                street_list.append('N/A')

        self.data['street'] = street_list
        
        return self.data
    
    def clean_data(self):
        # Remove useless columns
        data_cleaned = self.data[self.data.section != 'N/A'][['PROBE_ID','LAT','LONG',\
                                                            'HEADING',\
                                                            'SPEED',\
                                                            'PROBE_DATA_PROVIDER',\
                                                            'LOCAL_TIME',\
                                                            'coord',\
                                                            'section','street']]
        
    
    def probe_count(self, section):
        rht_uniq_probe = data_cleaned.loc[data_cleaned['section'] == section].PROBE_ID.unique()

        

In [91]:
analysis = Analyze_Data(data, section_shapefile_list, street_shapefile_list)
data_section = analysis.get_street()

In [92]:
data_section

,PROBE_ID,SAMPLE_DATE,LAT,LONG,HEADING,SPEED,PROBE_DATA_PROVIDER,X,Y,LOCAL_TIME,coord,section,street
480,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:00,33.615262,-117.927329,34.0,14.00000,CONSUMER14,413977.562793,3.719884e+06,2018-07-03 17:00:00 -0700,POINT (-117.927329 33.615262),right_half_tri,N/A
479,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:01,33.615269,-117.927325,19.0,12.00000,CONSUMER14,413977.940815,3.719885e+06,2018-07-03 17:00:01 -0700,POINT (-117.927325 33.615269),right_half_tri,N/A
445,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:02,33.615287,-117.927314,19.0,17.00000,CONSUMER14,413978.979134,3.719886e+06,2018-07-03 17:00:02 -0700,POINT (-117.927314 33.615287),right_half_tri,N/A
494,6c247e2bf8f447b2b31b7450261a1e3c,2018-07-04 00:00:03,33.618990,-117.930160,189.0,4.00000,CONSUMER21,413718.655373,3.720299e+06,2018-07-03 17:00:03 -0700,POINT (-117.93016 33.61899),right_half_tri,newport
2276,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:03,33.615319,-117.927299,18.0,20.00000,CONSUMER14,413980.402429,3.719890e+06,2018-07-03 17:00:03 -0700,POINT (-117.927299 33.615319),right_half_tri,N/A
1383,c9dae7a716114ece8f0395e27ab88668,2018-07-04 00:00:04,33.607480,-117.924130,101.0,13.00000,CONSUMER21,414266.619361,3.719018e+06,2018-07-03 17:00:04 -0700,POINT (-117.92413 33.60748),rest_of_new_port,w_balboa
2275,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:04,33.615371,-117.927271,18.0,24.00000,CONSUMER14,413983.051561,3.719896e+06,2018-07-03 17:00:04 -0700,POINT (-117.927271 33.615371),right_half_tri,N/A
493,ce9276c2866c4597b408d6f6222eed63,2018-07-04 00:00:04,33.618230,-117.919770,309.0,6.00000,CONSUMER21,414681.711915,3.720207e+06,2018-07-03 17:00:04 -0700,POINT (-117.91977 33.61823),N/A,N/A
2274,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:05,33.615423,-117.927240,18.0,27.00000,CONSUMER14,413985.978989,3.719902e+06,2018-07-03 17:00:05 -0700,POINT (-117.92724 33.615423),right_half_tri,N/A
2273,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:06,33.615488,-117.927215,18.0,30.00000,CONSUMER14,413988.362730,3.719909e+06,2018-07-03 17:00:06 -0700,POINT (-117.927215 33.615488),right_half_tri,N/A


## Notes
### For unique probe count
- Options:
- 'right_half_tri'
- 'left_half_tri'
- 'island'
- 'rest_of_new_port'